<a href="https://colab.research.google.com/github/hamzazenbi/MyApplication/blob/master/practica8_adicional1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Práctica 8. Ejercicio adicional 1: Preprocesado de datos

La calidad de los datos y la cantidad de información relevante que dichos datos contienen son factores clave a la hora de que un algoritmo de aprendizaje sea capaz de aprender. En este notebook se ven distintas técnicas de preprocesado de datos y su impacto a la hora de entrenar modelos. 

## 0. Carga de librerías y del dataset 

Para este notebook vamos a utilizar el dataset del Vino, otro dataset open-source que está disponible en el [repositorio UCI](https://archive.ics.uci.edu/ml/datasets/wine). Este dataset consiste en 178 muestras de vinos y 13 descriptores de distintas propiedades químicas. 

Usando pandas vamos a descargar directamente dicho dataset. También definimos en la siguiente celda el nombre de las columnas.

In [0]:
import pandas as pd
import numpy as np

In [0]:
df_wine = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data',
                     header=None)
df_wine.columns = ['Class label', 'Alcohol', 'Malid acid', 'Ash', 'Alcalinity of ash', 
                  'Magnesium', 'Total phenols','Flavanoids','Nonflavanoid phenols', 
                  'Proanthocyanins','Color intensity','Hue','OD280/OD315 of diluted wines',
                  'Proline']

A continuación podemos ver las categorías de vinos de nuestro dataset.

In [0]:
print('Class labels',np.unique(df_wine['Class label']))

##### Pregunta
¿Cuántas clases hay?

La siguiente instrucción muestra las primeras filas del dataset.

In [0]:
df_wine.head()

##### Ejercicio
La variable `df_wine` contiene tanto las etiquetas como los descriptores, separa dicha variable en las variables `X` e `y` como hemos hecho en otras ocasiones. Date cuenta que en este caso la etiqueta no la proporciona la última columna sino la primera, y que los descriptores van desde la columna 1 hasta la última. 

In [0]:
X = df_wine.values[:,1:]
y = df_wine.values[:,0]

La siguiente celda debería producir los siguientes resultados:

| Expresión | Resultado |
|---|---|
| X[0] | [  1.42300000e+01,1.71000000e+00,2.43000000e+00,1.56000000e+01,1.27000000e+02,2.80000000e+00   3.06000000e+00,2.80000000e-01,2.29000000e+00,5.64000000e+00, 1.04000000e+00,3.92000000e+00,1.06500000e+03] |
| y[5] | 1.0 |

In [0]:
print(X[0])
print(y[5])

##### Ejercicio
Como hacemos de manera habitual vamos a partir el dataset en los conjuntos de entrenamiento y test. Utiliza el 30% para el conjunto de entrenamiento y usa como `random_state` el valor 0.

In [0]:
X_train, X_test, y_train, y_test = ????

## 1. Escalando los descriptores

El escalado de los descriptores es un paso habitual de preprocesamiento. La mayoría de algoritmos de aprendizaje aprenden mucho mejor cuando los descriptores están en la misma escala. 

La importancia de escalar los descriptores se puede ilustrar con el siguiente ejemplo. Suponed que tenemos dos descriptores donde un descriptor se mide en una escala de 1 a 10 y otra se mide en una escala 1 a 100000. Cuando pensamos en el error cuadrático, es intuitivo que los algoritmos de optimización ajustarán los pesos de acuerdo a los errores producidos en el segundo descriptor. Otro ejemplo sería en el algoritmo KNN con la medida Euclídea, donde el cálculo de las distancias va a estar dominado por el segundo descriptor.

Existen dos aproximaciones distintas para escalar los descriptores: la normalización y la estandarización.

### 1.1. Normalización

La normalización se refiere al proceso de reescalar los descriptores en el rango $[0,1]$. Para llevar a cabo este reescalado se puede aplicar el escalado min-max a cada columna de un descriptor. En concreto para calcular este valor usamos la siguiente fórmula:
$$x_{norm}^{(i)} = \frac{x^{(i)}-x_{min}}{x_{max}-x_{min}}$$
donde $x_{norm}^{(i)}$ es el nuevo valor de la instancia $i$ del dataset para un descriptor, $x^{(i)}$ es el valor original de la instancia para ese descriptor, $x_{min}$ es el menor valor que toma ese descriptor para todas las instancias del dataset, y $x_{max}$ es el mayor valor que toma ese descriptor para todas las instancias del dataset.

Este procedimiento está implementado en sklearn y puede usarse del siguiente modo. Importamos la librería y definimos un objeto de la clase `MinMaxScaler`.

In [0]:
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()

Llevamos a cabo el reescalado tanto del conjunto de entrenamiento utilizando la función `fit_transform` del objeto `mms`, y luego reescalamos el conjunto de test utilizando la función `transform`. Esto se hace para que el reescalado del conjunto de test utilice los mismos valores de reescalado que el conjunto de entrenamiento.

In [0]:
X_train_norm = mms.fit_transform(X_train)
X_test_norm = mms.transform(X_test)

### 1.2. Estandarización

La normalización es una técnica útil cuando necesitamos que los nuevos valores de los descriptores estén en un intervalo, la estandarización puede ser más útil para los algoritmos de aprendizaje. La razón es que muchos algoritmos como la regresión logística, SVMs o las redes neuronales inicializan los pesos a 0 o a valores aleatoreos cercanos a 0. Usando la estandarización se consiguen centrar las columnas de descriptores con media 0 y desviación típica 1 de modo que las columnas de descriptores toman la forma de una distribución normal lo que hace que se aprendan de manera más sencilla los pesos. 

El proceso de estandarización viene dado por la siguiente fórmula:
$$x^{(i)}_{std} = \frac{x^{(i)}-\mu_x}{\sigma_x}$$
donde $\mu_x$ es la media de la muestra para cada columna de descriptores, $\sigma_x$ es la desviación típica, y $x^{(i)}_{std}$ es el valor calculado a partir del original $x^{(i)}$.

La siguiente tabla muestra la diferencia entre los valores normalizados y estandarizados en un dataset que contiene los números del 0 al 5. 

| Entrada | Estandarizado | Normalizado |
| --- | --- | --- |
| 0.0 | -1.33 | 0.0 |
| 1.0 | -0.8 | 0.2 |
| 2.0 | -0.26 | 0.4 |
| 3.0 | 0.26 | 0.6 |
| 4.0 | 0.8 | 0.8 |
| 5.0 | 1.33 | 1.0 |

Al igual que para la normalización, sklearn también implementa la estandarización y se utiliza del mismo modo.

In [0]:
from sklearn.preprocessing import StandardScaler
stdsc = StandardScaler()
X_train_std = stdsc.fit_transform(X_train)
X_test_std = stdsc.transform(X_test)

## 2. Seleccionando descriptores relevantes

Cuando un modelo funciona mucho mejor en el conjunto de entrenamiento que en el de test, estos es un claro caso de _sobreajuste_. Es decir, que el modelo ha ajustado sus parámetros para funcionar bien en el conjunto de entrenamiento pero no generaliza bien a datos reales. Existen distintas técnicas para reducir el sobreajuste, la más sencilla pero también más complicada de llevar a cabo consiste en conseguir más datos para entrenar el modelo; esto en muchas ocasiones no es posible. En este apartado vamos a ver otra técnica que consiste en reducir el sobreajuste que consiste en utilizar la reducción de dimensionalidad. 

Ya vimos que existen dos tipos de técnicas para reducir la dimensionalidad: la selección de descriptores y la extracción de descriptores. En este apartado nos vamos a centrar en el uso del método de selección secuencial hacia atrás. Este algoritmo no está implementado por defecto en sklearn, pero lo tienes disponible en el fichero sbs.py. Vamos a ver como funciona nuestro selector de descriptores utilizando el clasificador KNN. Comentamos cargando las librerías necesarias. 

In [0]:
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
from sbs import SBS
%matplotlib inline

Construimos el clasificador.

In [0]:
knn = KNeighborsClassifier(n_neighbors=2)

Construimos nuestro selector de descriptores y le indicamos que al menos tiene que tomar 1 descriptor. El proceso consiste en construir una instancia de la clase `SBS` donde le indicamos el clasificador y el mínimo número de descriptores, y luego entrenarla.

In [0]:
sbs= SBS(knn,k_features=1)
sbs.fit(X_train_std,y_train)

El objeto de la clase sbs almacena las puntuaciones de los mejores subconjuntos de descriptores en cada paso utilizando una parte del conjunto de test como conjunto de validación, así que podemos mostrar la precisión del clasificador para los distintos subconjuntos.

In [0]:
k_feat = [len(k) for k in sbs.subsets_]
plt.plot(k_feat,sbs.scores_,marker='o')
plt.ylim([0.7,1.1])
plt.ylabel('Accuracy')
plt.xlabel('Numero descriptores')
plt.grid()
plt.show()

Como se puede ver en la figura anterior la accuracy del algoritmo KNN aumenta al reducir el número de descriptores lo cual se debe a la maldición de la dimensionalidad. Como se puede ver en la gráfica, el algoritmo KNN alcanza una accuracy del 100% utilizando entre 5 y 11 descriptores. 

Podemos tomar los 5 descriptores más relevantes y ver cuáles son. Como estamos utilizando la eliminación hacia atrás para acceder a los 5 descriptores más relevantes tenemos que acceder al campo `subsets_` (que es una lista) e indicalre el índice que sería 13 (número total de descriptores) - 5 (número de descriptores con los que nos quedamos).

In [0]:
k5 = list(sbs.subsets_[8]) 
print(df_wine.columns[1:][k5])

Vamos ahora a evaluar el rendimiento del clasificador KNN en el conjunto de test. Primero lo consideramos sin estandarizar. 

In [0]:
knn.fit(X_train,y_train)
print('Training accuracy:', knn.score(X_train,y_train))
print('Test accuracy:', knn.score(X_test,y_test))

A continuación utilizando el dataset normalizado.

In [0]:
knn.fit(X_train_norm,y_train)
print('Training accuracy:', knn.score(X_train_norm,y_train))
print('Test accuracy:', knn.score(X_test_norm,y_test))

A continuación utilizando el dataset estandarizado.

In [0]:
knn.fit(X_train_std,y_train)
print('Training accuracy:', knn.score(X_train_std,y_train))
print('Test accuracy:', knn.score(X_test_std,y_test))

Por último considerando solo los 5 descriptores más relevantes. 

In [0]:
knn.fit(X_train_std[:,k5],y_train)
print('Training accuracy:', knn.score(X_train_std[:,k5],y_train))
print('Test accuracy:', knn.score(X_test_std[:,k5],y_test))

##### Pregunta
¿Qué ha ocurrido al estandarizar los datos? ¿y al normalizarlos? ¿Ha mejorado la accuracy de nuestro clasificador? ¿Qué ha ocurrido al utilizar los 5 descriptores más importantes?

## 3. Viendo la importancia de los descriptores 

Una técnica que también resulta útil para seleccionar descriptores relevantes a partir de un dataset consiste en utilizar [_random forests_](http://scikit-learn.org/stable/modules/ensemble.html#forest) un método en el cual se entrenan varios árboles de decisión y se organiza una votación entre esos modelos para decidir la predicción.

Utilizando los random forest podemos ver la importancia de cada descriptor usando el atributo `feature_importances_` que está accesible después de entrenar uno de estos clasificadores. Ejecutando el siguiente código se entrenará uno de estos modelos utilizando 1000 árboles de decisión y se obtendrá un ranking de los 13 descriptores. 

Comenzamos cargando las libreráis necesarias.

In [0]:
from sklearn.ensemble import RandomForestClassifier

Obtenemos el nombre de cada descriptor.

In [0]:
etiquetas_descriptores = df_wine.columns[1:]

Construimos el _random forest_.

In [0]:
forest = RandomForestClassifier(n_estimators=1000,random_state=0,n_jobs=-1)

Entrenamos el modelo. Notar que usamos directamente el dataset sin estandarizar, esto es debido a que los árboles de decisión no están afectados por la escala de los atributos. 

In [0]:
forest.fit(X_train,y_train)

Obtenemos la relevancia de cada descriptor y los índices para ordenar de mayor a menor dichas relevancias.

In [0]:
relevancias = forest.feature_importances_
indices = np.argsort(relevancias)[::-1]

Por último mostramos la importancia de cada descriptor.

In [0]:
for f in range(X_train.shape[1]):
    print("%2d) %-*s %f" % (f+1,30,etiquetas_descriptores[indices[f]],relevancias[indices[f]]))

##### Pregunta
¿Cuál es el descriptor más relevante?

Vamos a mostrar un gráfico con la relevancia de cada descriptor. 

In [0]:
plt.title('Relevancia de los descriptores')
plt.bar(range(X_train.shape[1]),relevancias[indices],color='lightblue',align='center')
plt.xticks(range(X_train.shape[1]),etiquetas_descriptores[indices],rotation=90)
plt.xlim([-1,X_train.shape[1]])
plt.tight_layout()
plt.show()